# Testing spaCy

In [ ]:
import sys
from tqdm import tqdm_notebook
import tqdm
import spacy
from spacy import displacy

In [ ]:
sys.executable

In [ ]:
# Load English tokenizer, tagger, parser, NER and word vectors
nlp_en = spacy.load('en')

# Load Portuguese tokenizer, tagger, parser, NER and word vectors
nlp_pt = spacy.load('pt')

In [ ]:
# Colors for the displacy entity visualizer
colors = {'ORG': '#bc8825',
          'PER': '#5df322',
          'PERSON': '#5df322',
          'DATE': '#3399ff',
          'Norp': '#ec134b',
          'MISC': '#6f6f6f',
          'LOC': '#ff81b0'
          }

# Color and entity options for the displacy visualizer
options = {'ents': ['ORG', 'PER', 'PERSON', 'DATE', 'Norp', 'MISC', 'LOC'], 'colors': colors}

### Example english short text

In [ ]:
text = (u"When Sebastian Thrun started working on self-driving cars at "
        u"Google in 2007, few people outside of the company took him "
        u"seriously. “I can tell you very senior CEOs of major American "
        u"car companies would shake my hand and turn away because I wasn’t "
        u"worth talking to,” said Thrun, now the co-founder and CEO of "
        u"online higher education startup Udacity, in an interview with "
        u"Recode earlier this week.")

In [ ]:
doc = nlp_en(text)

In [ ]:
# Find named entities, phrases and concepts
for entity in doc.ents:
    print(entity.text, entity.label_)

In [ ]:
# Only get the people entities
for entity in doc.ents:
    if 'PER' in entity.label_:
        print(entity.text)

In [ ]:
# Visualize the entities of the text in the (beautiful :) ) displacy visualizer
displacy.render(doc, style='ent', jupyter=True, options=options)

In [ ]:
# Determine semantic similarities
doc1 = nlp_en(u"my fries were super gross")
doc2 = nlp_en(u"such disgusting fries")
similarity = doc1.similarity(doc2)
print(doc1.text, doc2.text, similarity)

### Example portuguese short text

In [ ]:
text = (u"Quando as revelações atingem realidades que nos são próximas, é mais difícil "
        u"analisá-las com o distanciamento e a frieza com que analisamos outras. O caso "
        u"do Football Leaks – que, ainda por cima, mexe com o tão nacionalmente adorado "
        u"futebol – é um paradigma disso mesmo, com a animosidade a sobrepor-se à racionalidade, "
        u"e os sentimentos de invasão e traição a sobreporem-se ao eventual valor social, "
        u"político e institucional das revelações. Se é Rui Pinto o homem responsável pelo "
        u"Football Leaks, não será ele uma espécie de Julian Assange do futebol mundial? E digo "
        u"Assange para usar um nome concreto que poderia ser qualquer outro. No fundo, não serão "
        u"os Football Leaks tão saudáveis para a justiça no desporto como foram Wikileaks, Panama "
        u"Papers, Paradise Papers ou outras libertações de documentos do género? O que distingue "
        u"este whistleblower de outros?")

In [ ]:
doc = nlp_pt(text)

In [ ]:
# Find named entities, phrases and concepts
for entity in doc.ents:
    print(entity.text, entity.label_)

In [ ]:
# Only get the people entities
for entity in doc.ents:
    if 'PER' in entity.label_:
        print(entity.text)

In [ ]:
# Visualize the entities of the text in the (beautiful :) ) displacy visualizer
displacy.render(doc, style='ent', jupyter=True, options=options)